In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
#!unzip -q "/content/gdrive/My Drive/hvc_data.zip"

replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace resized/63.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
%tensorflow_version 1.x

import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.layers import AveragePooling2D, Input, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, Sequential
from keras.datasets import cifar10
import numpy as np
import os

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [5]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15,random_state=1)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=False, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])        
        if self.augmentation is not None:
            images = self.augmentation.flow(images, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

In [0]:
#x,y = train_gen.__getitem__(0)


In [0]:
# create train and validation data generators
#train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [32]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [45]:

backbone = InceptionV3(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

'''backbone = Sequential()
backbone.add(Conv2D(32, 3, 3, border_mode ="same", input_shape=(224,224, 3)))
backbone.add(Activation("relu"))
backbone.add(MaxPooling2D(pool_size=(2, 2)))

backbone.add(Conv2D(64, 2, 2, border_mode ="same"))
backbone.add(Activation("relu"))
backbone.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th'))'''


neck = backbone.output
#neck = BatchNormalization()
neck = Flatten(name="flatten")(neck)
#neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(1024, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    #neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_288 (Conv2D)             (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 111, 111, 32) 96          conv2d_288[0][0]                 
__________________________________________________________________________________________________
activation_288 (Activation)     (None, 111, 111, 32) 0           batch_normalization_283[0][0]    
____________________________________________________________________________________________

In [0]:
#freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [46]:
'''losses = {
 	  "gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
  	"age_output": "categorical_crossentropy",
  	"weight_output": "categorical_crossentropy",
}'''
loss_weights = {"gender_output": 1, "image_quality_output": 0.5, "age_output": 0.7, "weight_output":0.8,"footwear_output":0.75,
                "pose_output":0.8,
                "emotion_output":0.2
                ,"bag_output":0.8}
opt = SGD(lr=0.003)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",   
    loss_weights = loss_weights,
    metrics=["accuracy"]
)
#model.summary()

'''for layer in model.layers: 
  layer.trainable = False'''

'for layer in model.layers: \n  layer.trainable = False'

In [0]:
#model.fit(train_gen,validation_data=valid_gen, epochs=10)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=100,
    verbose=1
)

Epoch 1/100
360/360 [==============================] - 104s 290ms/step - loss: 7.8208 - gender_output_loss: 1.1520 - image_quality_output_loss: 1.4204 - age_output_loss: 1.9881 - weight_output_loss: 1.4613 - bag_output_loss: 1.3398 - footwear_output_loss: 1.4666 - pose_output_loss: 1.2603 - emotion_output_loss: 1.0896 - gender_output_acc: 0.5349 - image_quality_output_acc: 0.5198 - age_output_acc: 0.3638 - weight_output_acc: 0.6006 - bag_output_acc: 0.5246 - footwear_output_acc: 0.4737 - pose_output_acc: 0.5953 - emotion_output_acc: 0.7065 - val_loss: 13.7447 - val_gender_output_loss: 3.2974 - val_image_quality_output_loss: 2.0234 - val_age_output_loss: 3.5602 - val_weight_output_loss: 2.1158 - val_bag_output_loss: 2.0116 - val_footwear_output_loss: 2.5545 - val_pose_output_loss: 1.7044 - val_emotion_output_loss: 1.8107 - val_gender_output_acc: 0.4380 - val_image_quality_output_acc: 0.5549 - val_age_output_acc: 0.3221 - val_weight_output_acc: 0.6174 - val_bag_output_acc: 0.5338 - val_f

In [0]:
res = model.evaluate_generator(valid_gen,verbose=1)
dict(zip(model.metrics_names,res))

31/31 [==============================] - 5s 169ms/step


{'age_output_acc': 0.3911290322580645,
 'age_output_loss': 1.390031668447679,
 'bag_output_acc': 0.5766129032258065,
 'bag_output_loss': 0.8905611576572541,
 'emotion_output_acc': 0.7116935483870968,
 'emotion_output_loss': 0.8983784644834457,
 'footwear_output_acc': 0.5725806451612904,
 'footwear_output_loss': 0.9042290987506989,
 'gender_output_acc': 0.6285282258064516,
 'gender_output_loss': 0.6431156262274711,
 'image_quality_output_acc': 0.561491935483871,
 'image_quality_output_loss': 0.9451956518234745,
 'loss': 3.9530511517678537,
 'pose_output_acc': 0.6204637096774194,
 'pose_output_loss': 0.8776767503830695,
 'weight_output_acc': 0.6476814516129032,
 'weight_output_loss': 0.9627561972987267}